# Build LSOA data for England and Wales

### Other potential data sources
 * postcode to various hierarchies
   * https://geoportal.statistics.gov.uk/datasets/postcode-to-output-area-hierarchy-with-classifications-august-2020-lookup-in-the-uk
 * oa to lsoa / msoa / region etc
   * https://geoportal.statistics.gov.uk/datasets/output-area-to-lower-layer-super-output-area-to-middle-layer-super-output-area-to-local-authority-district-december-2020-lookup-in-england-and-wales/data

## Relationships within the tables

 * postcodes have many to one relationships between all other fields
 * wz (work zones) and oa (output areas) have a many to many relationship
 * oa and lsoa (lower layer super output areas) have a many to one relationship
 * lsoa and msoa (middle layer super output areas) have a many to one relationship
 * msoa and lad (local authority districts) have a many to one relationship
 * lad and rgn (regions) have a many to one relationship
 
```
pc --> oa11cd --> ladcd
   |          |\> lsoa11cd --> msoa11cd
   |          |            |\> soac11cd
   |          |            |\> laccd
   |          |             \> rgn20cd
   |           \> oac11cd
   |\> wz11cd 
   |\> wzc11cd
    \> lsoa11cd (duplicated for convenience)
```

## Step 1: Fetch the source datasets from their respective URLs

In [2]:
! wget -O imd.xlsx https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/833973/File_2_-_IoD2019_Domains_of_Deprivation.xlsx
! wget -O rural_urban.csv http://geoportal1-ons.opendata.arcgis.com/datasets/276d973d30134c339eaecfc3c49770b3_0.csv
! wget -O townsend.csv http://s3-eu-west-1.amazonaws.com/statistics.digitalresources.jisc.ac.uk/dkan/files/Townsend_Deprivation_Scores/Scores/Scores-%202011%20UK%20LSOA.csv
! wget -O oa_to_rgn.csv https://opendata.arcgis.com/datasets/65664b00231444edb3f6f83c9d40591f_0.csv
! wget -O pc_to_lsoa.zip https://www.arcgis.com/sharing/rest/content/items/83300a9b0e63465fabee3fddd8fbd30e/data
! unzip -o pc_to_lsoa.zip
! wget -O lsoa_to_utla.csv https://opendata.arcgis.com/datasets/9f4c270148014f20bf24abff9a7aef62_0.csv
! wget -O density.zip https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fpopulationandmigration%2fpopulationestimates%2fdatasets%2flowersuperoutputareapopulationdensity%2fmid2019sape22dt11/sape22dt11mid2019lsoapopulationdensity.zip
! unzip -o density.zip

--2021-01-25 15:58:18--  https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/833973/File_2_-_IoD2019_Domains_of_Deprivation.xlsx
Resolving assets.publishing.service.gov.uk (assets.publishing.service.gov.uk)... 151.101.16.144, 2a04:4e42:4::144
Connecting to assets.publishing.service.gov.uk (assets.publishing.service.gov.uk)|151.101.16.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4224939 (4.0M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘imd.xlsx’

imd.xlsx            100%[===================>]   4.03M  21.8MB/s    in 0.2s    

2021-01-25 15:58:18 (21.8 MB/s) - ‘imd.xlsx’ saved [4224939/4224939]

--2021-01-25 15:58:19--  http://geoportal1-ons.opendata.arcgis.com/datasets/276d973d30134c339eaecfc3c49770b3_0.csv
Resolving geoportal1-ons.opendata.arcgis.com (geoportal1-ons.opendata.arcgis.com)... 3.225.243.7, 3.214.85.175, 3.216.65.78, ...
Connecting to geoportal1-ons.opendata.arc

## Step 2: load the various source tables

In [3]:
import numpy as np
import pandas as pd

pc_df = pd.read_csv('NSPCL_MAY20_UK_LU.csv')

oa_to_rgn_df = pd.read_csv('oa_to_rgn.csv')
oa_to_rgn_df = oa_to_rgn_df.rename(columns={k:k.lower() for k in oa_to_rgn_df.columns})

# imd_df = pd.read_excel('imd.xlsx', sheet_name=1)
# imd_df = imd_df.rename(columns={"LSOA code (2011)": "lsoa11cd",
#                                 "LSOA name (2011)": "lsoa11nm",
#                                 "Local Authority District code (2019)": "lad19cd",
#                                 "Local Authority District name (2019)": "lad19nm",
#                                 "Index of Multiple Deprivation (IMD) Rank": "imd_rank",
#                                 "Index of Multiple Deprivation (IMD) Decile": "imd_decile"})

imd_df = pd.read_excel('imd.xlsx', sheet_name=1)
imd_df = imd_df.rename(columns={"LSOA code (2011)": "lsoa11cd",
                                "LSOA name (2011)": "lsoa11nm",
                                "Local Authority District code (2019)": "lad19cd",
                                "Local Authority District name (2019)": "lad19nm",
                                "Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)": "imd_rank",
                                "Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs)": "imd_decile",
                                "Income Rank (where 1 is most deprived)": "imd_income_rank",
                                "Income Decile (where 1 is most deprived 10% of LSOAs)": "imd_income_decile",
                                "Employment Rank (where 1 is most deprived)": "imd_employment_rank",
                                "Employment Decile (where 1 is most deprived 10% of LSOAs)": "imd_employment_decile",
                                "Education, Skills and Training Rank (where 1 is most deprived)": "imd_education_rank",
                                "Education, Skills and Training Decile (where 1 is most deprived 10% of LSOAs)": "imd_education_decile",
                                "Health Deprivation and Disability Rank (where 1 is most deprived)": "imd_health_rank",
                                "Health Deprivation and Disability Decile (where 1 is most deprived 10% of LSOAs)": "imd_health_decile",
                                "Crime Rank (where 1 is most deprived)": "imd_crime_rank",
                                "Crime Decile (where 1 is most deprived 10% of LSOAs)": "imd_crime_decile",
                                "Barriers to Housing and Services Rank (where 1 is most deprived)": "imd_housing_rank",
                                "Barriers to Housing and Services Decile (where 1 is most deprived 10% of LSOAs)": "imd_housing_decile",
                                "Living Environment Rank (where 1 is most deprived)": "imd_living_rank",
                                "Living Environment Decile (where 1 is most deprived 10% of LSOAs)": "imd_living_decile"} )

tsend_df = pd.read_csv('./townsend.csv')
tsend_df = tsend_df.rename(columns={"GEO_CODE": "lsoa11cd", "TDS": "townsend_score", "quintile": "townsend_quintile"})
tsend_df = tsend_df.drop(columns=[c for c in tsend_df.columns if c not in ("lsoa11cd", "townsend_score", "townsend_quintile")])

ruc_df = pd.read_csv('./rural_urban.csv')
ruc_df = ruc_df.rename(columns={"LSOA11CD": "lsoa11cd", "RUC11CD": "ruc11cd", "RUC11": "ruc11nm"})
ruc_df = ruc_df.drop(columns=[c for c in ruc_df.columns if c not in ("lsoa11cd", "ruc11cd", "ruc11nm")])

lsoa_to_utla_df = pd.read_csv('lsoa_to_utla.csv')
lsoa_to_utla_df = lsoa_to_utla_df.rename(columns={k:k.lower() for k in lsoa_to_utla_df.columns})

density_df = pd.read_excel('SAPE22DT11-mid-2019-lsoa-population-density.xlsx', sheet_name=3, skiprows=4)
density_df = density_df.rename(columns={"LSOA Code": "lsoa11cd", "LSOA Name": "lsoa11nm",
                                        "Mid-2019 population": "population", "Area Sq Km": "area_sq_km", "People per Sq Km": "density"})
print(density_df.columns)


print("done!")

/home/ben/.venvs/pytorch37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['lsoa11cd', 'lsoa11nm', 'population', 'area_sq_km', 'density'], dtype='object')
done!


## Step 3: build tables as the postcode, oa11cd and lsoa11cd levels

In [4]:
# generate sub-tables from pc_df
# ------------------------------

# pc --> oa11cd --> ladcd
#    |          |\> lsoa11cd --> msoa11cd (--> rgn20cd)
#    |          |            |\> soac11cd
#    |          |            |\> laccd
#    |          |             \> rgn20cd
#    |           \> oac11cd
#    |\> wz11cd 
#    |\> wzc11cd
#     \> lsoa11cd (duplicated for convenience)

# create a postcode to oa11cd / lsoa11cd lookup table from pc to oacd table
print("pc_df")
print(pc_df.columns)
pc_lookup_df = pc_df[['pcd7', 'pcd8', 'pcds', 'oa11cd', 'lsoa11cd', 'lsoa11nm', 'wz11cd', 'wzc11cd', 'wzc11nm']]
print("pc_lookup_df")
print(len(pc_lookup_df))
pc_lookup_df = pc_lookup_df[~pc_lookup_df['pcd7'].isna()]
print(len(pc_lookup_df))
pc_lookup_df = pc_lookup_df[~pc_lookup_df['pcd8'].isna()]
print(len(pc_lookup_df))
pc_lookup_df = pc_lookup_df[~pc_lookup_df['pcds'].isna()]
print(len(pc_lookup_df))

# create an oa11cd to ladcd / lsoa11cd lookup table from pc to oacd table
print("pc_oa11cd_lookup_df")
pc_oa11cd_lookup_df = pc_df[['oa11cd', 'lsoa11cd', 'ladcd', 'ladnm', 'ladnmw', 'oac11cd', 'oac11nm']]
print(len(pc_oa11cd_lookup_df))
pc_oa11cd_lookup_df = pc_oa11cd_lookup_df[~pc_oa11cd_lookup_df['oa11cd'].isna()]
print(len(pc_oa11cd_lookup_df))
pc_oa11cd_lookup_df = pc_oa11cd_lookup_df.drop_duplicates()
print(len(pc_oa11cd_lookup_df))

# create an lsoa11cd to msoa11cd / soac11cd lookup table from pc to oacd table
print("pc_lsoa11cd_lookup_df")
pc_lsoa11cd_lookup_df = pc_df[['lsoa11cd', 'lsoa11nm', 'msoa11cd', 'msoa11nm', 'soac11cd', 'soac11nm', 'laccd', 'lacnm']]
print(len(pc_lsoa11cd_lookup_df))
pc_lsoa11cd_lookup_df = pc_lsoa11cd_lookup_df[~pc_lsoa11cd_lookup_df['lsoa11cd'].isna()]
print(len(pc_lsoa11cd_lookup_df))
pc_lsoa11cd_lookup_df = pc_lsoa11cd_lookup_df.drop_duplicates()
print(len(pc_lsoa11cd_lookup_df))
# print(pc_lsoa11cd_lookup_df[pc_lsoa11cd_lookup_df['lsoa11cd'].isin(('L99999999', 'M99999999'))])

# generate sub-tables from oa_to_rgn_df
# -------------------------------------

# create an oa11cd to lsoa11cd lookup table from pc to oacd table
print("oa_to_rgn_df")
print(oa_to_rgn_df.columns)
print("oa_oa11cd_lookup_df")
oa_oa11cd_lookup_df = oa_to_rgn_df[['oa11cd', 'lsoa11cd', 'lad20cd', 'lad20nm']]
print(len(oa_oa11cd_lookup_df))
oa_oa11cd_lookup_df = oa_oa11cd_lookup_df[~oa_oa11cd_lookup_df['oa11cd'].isna()]
print(len(oa_oa11cd_lookup_df))
oa_oa11cd_lookup_df = oa_oa11cd_lookup_df.drop_duplicates()
print(len(oa_oa11cd_lookup_df))

# create a lsoa11cd to msoa11cd / rgn20cd
print("oa_lsoa11cd_lookup_df")
oa_lsoa11cd_lookup_df = oa_to_rgn_df[['lsoa11cd', 'lsoa11nm', 'msoa11cd', 'msoa11nm', 'rgn20cd', 'rgn20nm']]
print(len(oa_lsoa11cd_lookup_df))
oa_lsoa11cd_lookup_df = oa_lsoa11cd_lookup_df[~oa_lsoa11cd_lookup_df['lsoa11cd'].isna()]
print(len(oa_lsoa11cd_lookup_df))
oa_lsoa11cd_lookup_df = oa_lsoa11cd_lookup_df.drop_duplicates()
print(len(oa_lsoa11cd_lookup_df))


# generate sub-tables for imd_df
# ------------------------------
print("imd_df")
print(imd_df.columns)
print("imd_lsoa11cd_lookup_df")
imd_lsoa11cd_lookup_df = imd_df[['lsoa11cd', 'lsoa11nm', 'lad19cd', 'lad19nm',
                                 'imd_rank', 'imd_decile',
                                 'imd_income_rank', 'imd_income_decile', 'imd_employment_rank',
                                 'imd_employment_decile', 'imd_education_rank', 'imd_education_decile',
                                 'imd_health_rank', 'imd_health_decile', 'imd_crime_rank',
                                 'imd_crime_decile', 'imd_housing_rank', 'imd_housing_decile',
                                 'imd_living_rank', 'imd_living_decile']]
print(len(imd_lsoa11cd_lookup_df))
imd_lsoa11cd_lookup_df = imd_lsoa11cd_lookup_df[~imd_lsoa11cd_lookup_df['lsoa11cd'].isna()]
print(len(imd_lsoa11cd_lookup_df))
imd_lsoa11cd_lookup_df = imd_lsoa11cd_lookup_df.drop_duplicates()
print(len(imd_lsoa11cd_lookup_df))


# generate sub-tables for tsend_df
# --------------------------------
# ['lsoa11cd', 'townsend_score', 'townsend_quintile']
print("tsend_df")
print(tsend_df.columns)
print("tsend_lsoa11cd_lookup_df")
tsend_lsoa11cd_lookup_df = tsend_df[['lsoa11cd', 'townsend_score', 'townsend_quintile']]
print(len(tsend_lsoa11cd_lookup_df))
tsend_lsoa11cd_lookup_df = tsend_lsoa11cd_lookup_df[~tsend_lsoa11cd_lookup_df['lsoa11cd'].isna()]
print(len(tsend_lsoa11cd_lookup_df))
tsend_lsoa11cd_lookup_df = tsend_lsoa11cd_lookup_df.drop_duplicates()
print(len(tsend_lsoa11cd_lookup_df))


# generate sub-tables for ruc_df
# ------------------------------
# ['lsoa11cd', 'ruc11cd', 'ruc11nm']
print("ruc_df")
print(ruc_df.columns)
print("ruc_lsoa11cd_lookup_df")
ruc_lsoa11cd_lookup_df = ruc_df[['lsoa11cd', 'ruc11cd', 'ruc11nm']]
print(len(ruc_lsoa11cd_lookup_df))
ruc_lsoa11cd_lookup_df = ruc_lsoa11cd_lookup_df[~ruc_lsoa11cd_lookup_df['lsoa11cd'].isna()]
print(len(ruc_lsoa11cd_lookup_df))
ruc_lsoa11cd_lookup_df = ruc_lsoa11cd_lookup_df.drop_duplicates()
print(len(ruc_lsoa11cd_lookup_df))


# generate sub-tables for lsoa_to_utla_df
# ---------------------------------------
print("lsoa_to_utla_df")
print(lsoa_to_utla_df.columns)
print(len(lsoa_to_utla_df['utla17cd'].unique()))
lsoa_utla_lsoa11cd_lookup_df = lsoa_to_utla_df[['lsoa11cd', 'lsoa11nm', 'utla17cd', 'utla17nm']]
print(len(lsoa_utla_lsoa11cd_lookup_df))
lsoa_utla_lsoa11cd_lookup_df = lsoa_utla_lsoa11cd_lookup_df[~lsoa_utla_lsoa11cd_lookup_df['lsoa11cd'].isna()]
print(len(lsoa_utla_lsoa11cd_lookup_df))
lsoa_utla_lsoa11cd_lookup_df = lsoa_utla_lsoa11cd_lookup_df.drop_duplicates()
print(len(lsoa_utla_lsoa11cd_lookup_df))


# generate sub-tables for density_df
# ----------------------------------
print("density_df")
print(density_df.columns)
density_lsoa11cd_lookup_df = density_df[['lsoa11cd', 'lsoa11nm', 'population', 'area_sq_km', 'density']]
print(len(density_lsoa11cd_lookup_df))
density_lsoa11cd_lookup_df = density_lsoa11cd_lookup_df[~density_lsoa11cd_lookup_df['lsoa11cd'].isna()]
print(len(density_lsoa11cd_lookup_df))
density_lsoa11cd_lookup_df = density_lsoa11cd_lookup_df.drop_duplicates()
print(len(density_lsoa11cd_lookup_df))


pc_df
Index(['pcd7', 'pcd8', 'pcds', 'dointr', 'doterm', 'usertype', 'oseast1m',
       'osnrth1m', 'oa11cd', 'oac11cd', 'oac11nm', 'wz11cd', 'wzc11cd',
       'wzc11nm', 'lsoa11cd', 'lsoa11nm', 'msoa11cd', 'msoa11nm', 'soac11cd',
       'soac11nm', 'ladcd', 'ladnm', 'ladnmw', 'laccd', 'lacnm'],
      dtype='object')
pc_lookup_df
2643728
2643728
2643728
2643728
pc_oa11cd_lookup_df
2643728
2633739
232040
pc_lsoa11cd_lookup_df
2643728
2633739
42621
oa_to_rgn_df
Index(['fid', 'oa11cd', 'lsoa11cd', 'lsoa11nm', 'msoa11cd', 'msoa11nm',
       'lad20cd', 'lad20nm', 'rgn20cd', 'rgn20nm'],
      dtype='object')
oa_oa11cd_lookup_df
181408
181408
181408
oa_lsoa11cd_lookup_df
181408
181408
34753
imd_df
Index(['lsoa11cd', 'lsoa11nm', 'lad19cd', 'lad19nm', 'imd_rank', 'imd_decile',
       'imd_income_rank', 'imd_income_decile', 'imd_employment_rank',
       'imd_employment_decile', 'imd_education_rank', 'imd_education_decile',
       'imd_health_rank', 'imd_health_decile', 'imd_crime_rank',
       '

## Step 4: Merge osa tables together and lsoa tables together, checking common fields for consistency

In [6]:
def check_consistency(msg, one, two):
    print("check_consistency()")
    mismatches = 0
    one = dict(sorted(one.values.tolist()))
    two = dict(sorted(two.values.tolist()))
    for k,v in one.items():
        if k in two and two[k] != v:
            mismatches += 1
    print("{}: {} mismatches".format(msg, mismatches))
    
def consolidate_fields(df, result, first, second):
    print("consolidate_fields({}, {}, {}):".format(result, first, second))
    tbl = df[[first, second]]
    print(len(tbl))
    tbl = tbl.dropna(how='any')
    print(len(tbl))
    tbl = tbl[tbl[first] != tbl[second]]
    if len(tbl) > 0:
        print("Unexpected: non-null entries mismatched")
        print(tbl)
    print(len(tbl))

    df[result] = df[first].where(df[first].notna(), df[second])
    df = df.drop(columns=[first, second])
    return df

# double check that oa1cd -> lsoa11cd mappings are in mutual agreement

check_consistency("checking oa11cd -> lsoa11cd consistency",
                  pc_oa11cd_lookup_df[['oa11cd', 'lsoa11cd']],
                  oa_oa11cd_lookup_df[['oa11cd', 'lsoa11cd']])

check_consistency("checking lsoa11cd consistency (pc, oa)",
                  pc_lsoa11cd_lookup_df[['lsoa11cd', 'lsoa11nm']],
                  oa_lsoa11cd_lookup_df[['lsoa11cd', 'lsoa11nm']])

check_consistency("checking lsoa11cd consistency (pc, imd)",
                  pc_lsoa11cd_lookup_df[['lsoa11cd', 'lsoa11nm']],
                  imd_lsoa11cd_lookup_df[['lsoa11cd', 'lsoa11nm']])

check_consistency("checking lsoa11cd consistency (pc, utla)",
                  pc_lsoa11cd_lookup_df[['lsoa11cd', 'lsoa11nm']],
                  lsoa_utla_lsoa11cd_lookup_df[['lsoa11cd', 'lsoa11nm']])

print('\noa11cd_lookup_df')
oa11cd_lookup_df = pd.merge(left_on='oa11cd', right_on='oa11cd',
                            left=pc_oa11cd_lookup_df, right=oa_oa11cd_lookup_df,
                            how='outer')
oa11cd_lookup_df = consolidate_fields(oa11cd_lookup_df, 'lsoa11cd', 'lsoa11cd_x', 'lsoa11cd_y')
print(oa11cd_lookup_df.columns)

print('\nlsoa11cd_lookup_df')
lsoa11cd_lookup_df = pd.merge(left_on='lsoa11cd', right_on='lsoa11cd',
                              left=pc_lsoa11cd_lookup_df, right=oa_lsoa11cd_lookup_df,
                              how='outer')
lsoa11cd_lookup_df = consolidate_fields(lsoa11cd_lookup_df, 'lsoa11nm', 'lsoa11nm_x', 'lsoa11nm_y')
lsoa11cd_lookup_df = consolidate_fields(lsoa11cd_lookup_df, 'msoa11cd', 'msoa11cd_x', 'msoa11cd_y')
lsoa11cd_lookup_df = consolidate_fields(lsoa11cd_lookup_df, 'msoa11nm', 'msoa11nm_x', 'msoa11nm_y')
print(lsoa11cd_lookup_df.columns)

print('\nlsoa11cd_lookup with imd')
print(len(lsoa11cd_lookup_df))
lsoa11cd_lookup_df = pd.merge(left_on='lsoa11cd', right_on='lsoa11cd',
                              left=lsoa11cd_lookup_df, right=imd_lsoa11cd_lookup_df,
                              how='outer')
print(len(lsoa11cd_lookup_df))
lsoa11cd_lookup_df = consolidate_fields(lsoa11cd_lookup_df, 'lsoa11nm', 'lsoa11nm_x', 'lsoa11nm_y')
print(lsoa11cd_lookup_df.columns)

print('\nlsoa11cd_lookup with tsend')
print(len(lsoa11cd_lookup_df))
lsoa11cd_lookup_df = pd.merge(left_on='lsoa11cd', right_on='lsoa11cd',
                              left=lsoa11cd_lookup_df, right=tsend_lsoa11cd_lookup_df,
                              how='outer')
print(len(lsoa11cd_lookup_df))
print(lsoa11cd_lookup_df.columns)

print('\nlsoa11cd_lookup_with_ruc')
print(len(lsoa11cd_lookup_df))
lsoa11cd_lookup_df = pd.merge(left_on='lsoa11cd', right_on='lsoa11cd',
                              left=lsoa11cd_lookup_df, right=ruc_lsoa11cd_lookup_df,
                              how='outer')
print(len(lsoa11cd_lookup_df))
print(lsoa11cd_lookup_df.columns)

print('\nlsoa11cd_lookup with lsoa_to_utla')
print(len(lsoa11cd_lookup_df))
lsoa11cd_lookup_df = pd.merge(left_on='lsoa11cd', right_on='lsoa11cd',
                              left=lsoa11cd_lookup_df, right=lsoa_utla_lsoa11cd_lookup_df,
                              how='outer')
print(len(lsoa11cd_lookup_df))
lsoa11cd_lookup_df = consolidate_fields(lsoa11cd_lookup_df, 'lsoa11nm', 'lsoa11nm_x', 'lsoa11nm_y')
print(lsoa11cd_lookup_df.columns)

print('\nlsoa11cd_lookup with density')
print(len(lsoa11cd_lookup_df))
lsoa11cd_lookup_df = pd.merge(left_on='lsoa11cd', right_on='lsoa11cd',
                              left=lsoa11cd_lookup_df, right=density_lsoa11cd_lookup_df.drop(columns=['lsoa11nm']),
                              how='outer')
print(len(lsoa11cd_lookup_df))
print(lsoa11cd_lookup_df.columns)


check_consistency()
checking oa11cd -> lsoa11cd consistency: 0 mismatches
check_consistency()
checking lsoa11cd consistency (pc, oa): 0 mismatches
check_consistency()
checking lsoa11cd consistency (pc, imd): 0 mismatches
check_consistency()
checking lsoa11cd consistency (pc, utla): 0 mismatches

oa11cd_lookup_df
consolidate_fields(lsoa11cd, lsoa11cd_x, lsoa11cd_y):
232075
181373
0
Index(['oa11cd', 'ladcd', 'ladnm', 'ladnmw', 'oac11cd', 'oac11nm', 'lad20cd',
       'lad20nm', 'lsoa11cd'],
      dtype='object')

lsoa11cd_lookup_df
consolidate_fields(lsoa11nm, lsoa11nm_x, lsoa11nm_y):
42621
34753
0
consolidate_fields(msoa11cd, msoa11cd_x, msoa11cd_y):
42621
34753
0
consolidate_fields(msoa11nm, msoa11nm_x, msoa11nm_y):
42621
34753
0
Index(['lsoa11cd', 'soac11cd', 'soac11nm', 'laccd', 'lacnm', 'rgn20cd',
       'rgn20nm', 'lsoa11nm', 'msoa11cd', 'msoa11nm'],
      dtype='object')

lsoa11cd_lookup with imd
42621
42621
consolidate_fields(lsoa11nm, lsoa11nm_x, lsoa11nm_y):
42621
32844
0
Index(

## Step 6a: Export csv files for the consolidated postcode, oa11cd and lsoa11cd tables

In [ ]:
pc_lookup_df.to_csv('/home/ben/covid/lsoa3/pc_lookups.csv')
oa11cd_lookup_df.to_csv('/home/ben/covid/lsoa3/oa11cd_lookups.csv')
lsoa11cd_lookup_df.to_csv('/home/ben/covid/lsoa3/lsoa11cd_lookups.csv')
print("done!")

## Step 6b: Export the consolidated postcode, oa11cd and lsoa11cd tables as an ExeTera datastore

In [7]:
from exetera.core.session import Session
from exetera.core.persistence import try_str_to_int, try_str_to_float
print(pc_lookup_df.columns)
print(oa11cd_lookup_df.columns)
print(lsoa11cd_lookup_df.columns)

def fixed_string_fn(s, g, n, df):
    print('fixed_string_fn:', n)
    values = df[n].replace(np.nan, '').str.encode('utf-8')
    s.create_fixed_string(g, n, values.map(len).max()).data.write(values.to_list())
    
def indexed_string_fn(s, g, n, df):
    print('indexed_string_fn:', n)
    values = df[n].replace(np.nan, '').astype('str')
    s.create_indexed_string(g, n).data.write(values.to_list())

def rank_and_decile_fn(s, g, nr, nd, nv, df):
    print("rank_and_decile_fn:", nr, nd)
    valid = list()
    rank = list()
    decile = list()
    fvalid = s.create_numeric(g, nv, 'bool')
    frank = s.create_numeric(g, nr, 'int8')
    fdecile = s.create_numeric(g, nd, 'int32')
    for i in range(len(df[nr])):
        f, vr = try_str_to_int(df[nr][i])
        _, vd = try_str_to_int(df[nd][i])
        valid.append(f)
        rank.append(vr)
        decile.append(vd)
    fvalid.data.write(valid)
    frank.data.write(rank)
    fdecile.data.write(decile)
    
def townsend_fn(s, g, ns, nq, nv, df):
    print("townsend_fn:", ns, nq)
    valid = list()
    score = list()
    quintile = list()
    fvalid = s.create_numeric(g, nv, 'bool')
    fscore = s.create_numeric(g, ns, 'float32')
    fquintile = s.create_numeric(g, nq, 'int8')
    for i in range(len(df[ns])):
        f, vs = try_str_to_float(df[ns][i])
        _, vq = try_str_to_int(df[nq][i])
        valid.append(f)
        score.append(vs)
        quintile.append(vq)
    fvalid.data.write(valid)
    fscore.data.write(score)
    fquintile.data.write(quintile)

def numeric_fn(s, g, n, nv, df, map_fn, dtype):
    print('numeric_fn:', n)
    valid = list()
    data = list()
    fvalid = s.create_numeric(g, nv, 'bool')
    fdata = s.create_numeric(g, n, dtype)
    for i in range(len(df[n])):
        f, v = map_fn(df[n][i])
        valid.append(f)
        data.append(v)
    fvalid.data.write(valid)
    fdata.data.write(data)

def categorical_fn(s, g, n, df, dvals):
    print('categorical_fn:', n)
    values = df[n].replace(np.nan, '').astype('str')
    values = [dvals[v] for v in values]
    s.create_categorical(g, n, 'int8', dvals).data.write(values)
    
def create_categorical_dicts(df, n_cd, n_nm):
    codes = df[[n_cd, n_nm]].replace(np.nan, '').drop_duplicates().sort_values(by=n_cd).reset_index()
    cddict = {}
    nmdict = {}
    for i, r in codes.iterrows():
        cddict[r[n_cd]] = i
        nmdict[r[n_nm]] = i
    return cddict, nmdict

with Session() as s:
    dest = s.open_dataset('/home/ben/covid/lsoa3/geocodes_uk.hdf5', 'w', 'dest')

    d_pc = dest.create_group('postcode')
    fixed_string_fn(s, d_pc, 'pcd7', pc_lookup_df)
    fixed_string_fn(s, d_pc, 'pcd8', pc_lookup_df)
    fixed_string_fn(s, d_pc, 'pcds', pc_lookup_df)
    fixed_string_fn(s, d_pc, 'oa11cd', pc_lookup_df)
    fixed_string_fn(s, d_pc, 'lsoa11cd', pc_lookup_df)
    indexed_string_fn(s, d_pc, 'lsoa11nm', pc_lookup_df)
    fixed_string_fn(s, d_pc, 'wz11cd', pc_lookup_df)
    cddict, nmdict = create_categorical_dicts(pc_lookup_df, 'wzc11cd', 'wzc11nm')
    categorical_fn(s, d_pc, 'wzc11cd', pc_lookup_df, cddict)
    categorical_fn(s, d_pc, 'wzc11nm', pc_lookup_df, nmdict)

    d_oa = dest.create_group('osa11cd')
    fixed_string_fn(s, d_oa, 'oa11cd', oa11cd_lookup_df)
    fixed_string_fn(s, d_oa, 'lsoa11cd', oa11cd_lookup_df)
    fixed_string_fn(s, d_oa, 'ladcd', oa11cd_lookup_df)
    indexed_string_fn(s, d_oa, 'ladnm', oa11cd_lookup_df)
    indexed_string_fn(s, d_oa, 'ladnmw', oa11cd_lookup_df)
    cddict, nmdict = create_categorical_dicts(oa11cd_lookup_df, 'oac11cd', 'oac11nm')
    categorical_fn(s, d_oa, 'oac11cd', oa11cd_lookup_df, cddict)
    categorical_fn(s, d_oa, 'oac11nm', oa11cd_lookup_df, nmdict)
    
    d_lsoa = dest.create_group('lsoa11cd')
    fixed_string_fn(s, d_lsoa, 'lsoa11cd', lsoa11cd_lookup_df)
    indexed_string_fn(s, d_lsoa, 'lsoa11nm', lsoa11cd_lookup_df)
    cddict, nmdict = create_categorical_dicts(lsoa11cd_lookup_df, 'soac11cd', 'soac11nm')
    categorical_fn(s, d_lsoa, 'soac11cd', lsoa11cd_lookup_df, cddict)
    categorical_fn(s, d_lsoa, 'soac11nm', lsoa11cd_lookup_df, nmdict)
#     print(lsoa11cd_lookup_df['laccd'].unique())
#     cddict, nmdict = create_categorical_dicts(lsoa11cd_lookup_df, 'laccd', 'lacnm')
    fixed_string_fn(s, d_lsoa, 'msoa11cd', lsoa11cd_lookup_df)
    indexed_string_fn(s, d_lsoa, 'msoa11nm', lsoa11cd_lookup_df)
    fixed_string_fn(s, d_lsoa, 'lad19cd', lsoa11cd_lookup_df)
    indexed_string_fn(s, d_lsoa, 'lad19nm', lsoa11cd_lookup_df)
    fixed_string_fn(s, d_lsoa, 'utla17cd', lsoa11cd_lookup_df)
    indexed_string_fn(s, d_lsoa, 'utla17nm', lsoa11cd_lookup_df)
    fixed_string_fn(s, d_lsoa, 'rgn20cd', lsoa11cd_lookup_df)
    indexed_string_fn(s, d_lsoa, 'rgn20nm', lsoa11cd_lookup_df)
    rank_and_decile_fn(s, d_lsoa, 'imd_rank', 'imd_decile', 'imd_valid', lsoa11cd_lookup_df)
    rank_and_decile_fn(s, d_lsoa, 'imd_education_rank', 'imd_education_decile', 'imd_education_valid', lsoa11cd_lookup_df)
    rank_and_decile_fn(s, d_lsoa, 'imd_income_rank', 'imd_income_decile', 'imd_income_valid', lsoa11cd_lookup_df)
    rank_and_decile_fn(s, d_lsoa, 'imd_employment_rank', 'imd_employment_decile', 'imd_employment_valid', lsoa11cd_lookup_df)
    rank_and_decile_fn(s, d_lsoa, 'imd_health_rank', 'imd_health_decile', 'imd_health_valid', lsoa11cd_lookup_df)
    rank_and_decile_fn(s, d_lsoa, 'imd_crime_rank', 'imd_crime_decile', 'imd_crime_valid', lsoa11cd_lookup_df)
    rank_and_decile_fn(s, d_lsoa, 'imd_housing_rank', 'imd_housing_decile', 'imd_housing_valid', lsoa11cd_lookup_df)
    rank_and_decile_fn(s, d_lsoa, 'imd_living_rank', 'imd_living_decile', 'imd_living_valid', lsoa11cd_lookup_df)
    cddict, nmdict = create_categorical_dicts(lsoa11cd_lookup_df, 'ruc11cd', 'ruc11nm')
    categorical_fn(s, d_lsoa, 'ruc11cd', lsoa11cd_lookup_df, cddict)
    categorical_fn(s, d_lsoa, 'ruc11nm', lsoa11cd_lookup_df, nmdict)
    townsend_fn(s, d_lsoa, 'townsend_score', 'townsend_quintile', 'townsend_valid', lsoa11cd_lookup_df)
    numeric_fn(s, d_lsoa, 'population', 'population_valid', lsoa11cd_lookup_df, try_str_to_int, 'int32')
    numeric_fn(s, d_lsoa, 'area_sq_km', 'area_sq_km_valid', lsoa11cd_lookup_df, try_str_to_float, 'float32')
    numeric_fn(s, d_lsoa, 'density', 'density_valid', lsoa11cd_lookup_df, try_str_to_float, 'float32')
    

Index(['pcd7', 'pcd8', 'pcds', 'oa11cd', 'lsoa11cd', 'lsoa11nm', 'wz11cd',
       'wzc11cd', 'wzc11nm'],
      dtype='object')
Index(['oa11cd', 'ladcd', 'ladnm', 'ladnmw', 'oac11cd', 'oac11nm', 'lad20cd',
       'lad20nm', 'lsoa11cd'],
      dtype='object')
Index(['lsoa11cd', 'soac11cd', 'soac11nm', 'laccd', 'lacnm', 'rgn20cd',
       'rgn20nm', 'msoa11cd', 'msoa11nm', 'lad19cd', 'lad19nm', 'imd_rank',
       'imd_decile', 'imd_income_rank', 'imd_income_decile',
       'imd_employment_rank', 'imd_employment_decile', 'imd_education_rank',
       'imd_education_decile', 'imd_health_rank', 'imd_health_decile',
       'imd_crime_rank', 'imd_crime_decile', 'imd_housing_rank',
       'imd_housing_decile', 'imd_living_rank', 'imd_living_decile',
       'townsend_score', 'townsend_quintile', 'ruc11cd', 'ruc11nm', 'utla17cd',
       'utla17nm', 'lsoa11nm', 'population', 'area_sq_km', 'density'],
      dtype='object')
fixed_string_fn: pcd7
fixed_string_fn: pcd8
fixed_string_fn: pcds
fixed_string

### Ancillary code to check relationships

In [ ]:
def list_relationships(df, key_pairs):
# oa -> lsoa
    for kp in key_pairs:
        print(kp)
        sdf = df[kp]
        sdf = sdf.drop_duplicates()
        print(len(sdf))
        print(len(sdf[kp[0]].unique()))
        print(len(sdf[kp[1]].unique()))
        print(sdf[kp[0]].value_counts().unique())

print('pc_df')
list_relationships(pc_df,
                   (['oa11cd', 'oac11cd'], ['oa11cd', 'lsoa11cd'], ['oa11cd', 'msoa11cd'], ['oa11cd', 'ladcd'], ['oa11cd', 'soac11cd'],
                    ['oa11cd', 'laccd'], ['oa11cd', 'wz11cd'], ['oa11cd', 'wzc11cd'],
                    ['lsoa11cd', 'oac11cd'], ['lsoa11cd', 'msoa11cd'], ['lsoa11cd', 'ladcd'], ['lsoa11cd', 'soac11cd'],
                    ['lsoa11cd', 'laccd'], ['lsoa11cd', 'wz11cd'], ['lsoa11cd', 'wzc11cd'],
                    ['msoa11cd', 'ladcd'], ['msoa11cd', 'soac11cd'], ['ladcd', 'soac11cd']))

print('oa_to_rgn')
list_relationships(oa_to_rgn_df,
                   (['oa11cd', 'lsoa11cd'], ['oa11cd', 'lad20cd']))